# Experimento 5 
***
- Rede utilizada: Inception
- Conjunto de Dados: VinBigData
- Modelando uma para classificar problemas pulmonares em 11 classes distintas

### Importação dos pacotes

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Pré-processamento nos dados

In [2]:
# lendo os dados de um arquivo csv
dataframe = pd.read_csv('/content/drive/MyDrive/vinbigdata/train.csv')
# criando uma coluna com os caminhos relativos as imagens
dataframe['image_path'] = '/content/drive/MyDrive/vinbigdata/train/' + dataframe.image_id + '.jpg'

In [3]:
print('total de imagens disponíveis:', str(len(set(dataframe['image_path']))))

total de imagens disponíveis: 15000


In [4]:
# visualizando os casos disponíveis
dataframe['class_name'].value_counts()

No finding            31818
Aortic enlargement     7162
Cardiomegaly           5427
Pleural thickening     4842
Pulmonary fibrosis     4655
Nodule/Mass            2580
Lung Opacity           2483
Pleural effusion       2476
Other lesion           2203
Infiltration           1247
ILD                    1000
Calcification           960
Consolidation           556
Atelectasis             279
Pneumothorax            226
Name: class_name, dtype: int64

In [5]:
# removendo os casos não relativos a distúrbios pulmonares
dataframe = dataframe[dataframe.class_name != 'Aortic enlargement']
dataframe = dataframe[dataframe.class_name != 'Cardiomegaly']
dataframe = dataframe[dataframe.class_name != 'Other lesion']

In [6]:
# separando os casos rotulados como normais e anormais
normal_cases = dataframe[(dataframe.class_id == 14) & (dataframe.class_name == 'No finding')]
abnormal_cases = dataframe[(dataframe.class_id != 14) & (dataframe.class_name != 'No finding')]

print('total de dados após a filtração:', str(len(set(normal_cases['image_path'])) + len(set(abnormal_cases['image_path']))))

total de dados após a filtração: 13952


In [7]:
# removendo as imagens repetidas
normal_data = normal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )
abnormal_data = abnormal_cases[['image_path', 'class_name']].drop_duplicates(subset = 'image_path', )

# criando dataframes especifos com caminhos para as imagens e rótulos
normal_data['target'] = 'normal'
abnormal_data['target'] = 'abnormal'

In [8]:
print('quantidade de dados rotulados como normais:', len(normal_data))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 10606
quantidade de dados rotulados como anormais: 3346


In [9]:
# removendo 69% dos casos normais para balancear os dados
normal, _ = train_test_split(normal_data, test_size = 0.69, random_state = 42)

In [10]:
print('quantidade de dados rotulados como normais:', len(normal))
print('quantidade de dados rotulados como anormais:', len(abnormal_data))

quantidade de dados rotulados como normais: 3287
quantidade de dados rotulados como anormais: 3346


In [11]:
# concatenando os dataframes de casos normais e anormais
full_data = abnormal_data

In [12]:
# visualizando a quantidade de exemplos por classe disponíveis
full_data['class_name'].value_counts()

Pleural thickening    901
Pulmonary fibrosis    742
Lung Opacity          427
Nodule/Mass           339
Pleural effusion      328
Calcification         167
Infiltration          163
ILD                   152
Consolidation          59
Atelectasis            41
Pneumothorax           27
Name: class_name, dtype: int64

In [13]:
# separando os dados de treinamento e de teste
train_df, test_df = train_test_split(full_data, stratify = full_data['target'],
                                     test_size = 0.2, random_state = 42)

In [14]:
# separando os dados de validação dos dados de treinamento
train_df, validation_df = train_test_split(train_df, stratify = train_df['target'],
                                           test_size = 0.2, random_state = 42)

In [15]:
# visualizando a quantidade de dados
print('quantidade de imagens de treinamento:', len(train_df['image_path']))
print('quantidade de rótulos de treinamento:', len(train_df['target']))
print('quantidade de imagens de teste:', len(test_df['image_path']))
print('quantidade de rótulos de teste:', len(test_df['target']))
print('quantidade de imagens de validação:', len(validation_df['image_path']))
print('quantidade de rótulos de validação:', len(validation_df['target']))

quantidade de imagens de treinamento: 2140
quantidade de rótulos de treinamento: 2140
quantidade de imagens de teste: 670
quantidade de rótulos de teste: 670
quantidade de imagens de validação: 536
quantidade de rótulos de validação: 536


In [16]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      directory = '',
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

# normalizando as imagens de teste e validação
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                      dataframe = validation_df,
                                                      directory = '.', 
                                                      x_col = 'image_path',
                                                      y_col = 'class_name',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  directory = '.',
                                                  x_col = 'image_path',
                                                  y_col = 'class_name',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 2140 validated image filenames belonging to 11 classes.
Found 536 validated image filenames belonging to 11 classes.
Found 670 validated image filenames belonging to 11 classes.


### Preparando a rede neural convolucional

In [17]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-04-22 04:11:52--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 74.125.128.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  91.2MB/s    in 0.9s    

2021-04-22 04:11:53 (91.2 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [18]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), 
                                include_top = False, 
                                weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
pre_trained_model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed8'
for layer in pre_trained_model.layers:
    if layer.name == 'mixed8':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [19]:
x = layers.Flatten()(last_output)
# adicionando uma camada densa com 512 neurônios
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)      
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 2, activation = tf.nn.softmax)(x)      

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [20]:
# realizando a transferência de aprendizado com os pesos do experimento 3
model.load_weights('/content/drive/MyDrive/experimentos/v2.0-exp3-ds4/transferlearning_weights.hdf5', )

In [21]:
# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = model.get_layer('dropout')
last_output = last_layer.output

# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense(units = 11, activation = tf.nn.softmax)(last_output)      

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.0001), loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [22]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

lr_reduce = ReduceLROnPlateau(monitor = 'val_acc', factor=0.1, min_delta = 0.00001, patience = 5, verbose = 1)
# definindo um array de callbacks
callbacks = [checkpoint, lr_reduce]

In [23]:
# definindo as flags iniciais  
model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed8'
for layer in model.layers:
    if layer.name == 'mixed5':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [24]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 2140 // 32, 
                              validation_data = valid_generator, validation_steps = 536 // 32,
                              callbacks = callbacks, epochs = 20, use_multiprocessing = True,
                              workers = 8)

Epoch 1/20
66/66 [==============================] - 86s 1s/step - loss: 2.6816 - acc: 0.2089 - val_loss: 2.1027 - val_acc: 0.3066

Epoch 00001: val_acc improved from -inf to 0.30664, saving model to transferlearning_weights.hdf5
Epoch 2/20
66/66 [==============================] - 81s 1s/step - loss: 2.0343 - acc: 0.2814 - val_loss: 2.0575 - val_acc: 0.2734

Epoch 00002: val_acc did not improve from 0.30664
Epoch 3/20
66/66 [==============================] - 82s 1s/step - loss: 1.9101 - acc: 0.3216 - val_loss: 2.0814 - val_acc: 0.2988

Epoch 00003: val_acc did not improve from 0.30664
Epoch 4/20
66/66 [==============================] - 81s 1s/step - loss: 1.8275 - acc: 0.3421 - val_loss: 2.0554 - val_acc: 0.2871

Epoch 00004: val_acc did not improve from 0.30664
Epoch 5/20
66/66 [==============================] - 81s 1s/step - loss: 1.7582 - acc: 0.3737 - val_loss: 1.9787 - val_acc: 0.3125

Epoch 00005: val_acc improved from 0.30664 to 0.31250, saving model to transferlearning_weights.h

### Salvando o modelo

In [25]:
# carregando o melhor peso obtido para o modelo
best_model = model
best_model.load_weights('/content/transferlearning_weights.hdf5')

In [26]:
# salvando os dois modelos obtidos durante o treinamento
model.save('model1')
best_model.save('model2')

INFO:tensorflow:Assets written to: model1/assets
INFO:tensorflow:Assets written to: model2/assets


In [27]:
# carregando o melhor modelo para realização de testes de desempenho
model = tf.keras.models.load_model('/content/drive/MyDrive/experimentos/v2.0-exp3-ds4/model2')

In [28]:
model.evaluate(test_generator)

InvalidArgumentError: ignored

### Métricas de avaliação do modelo

In [ ]:
# carregando os dados de teste
for i in range(0, 42):
  (x1, y1) = test_generator[i]
  if i == 0:
    x, y = x1, y1
  else:
    x = np.concatenate((x, x1))
    y = np.concatenate((y, y1))

In [ ]:
# realizando a predição para os dados de teste
predict = model.predict(x)

In [ ]:
count_global = 0
for predicts in predict:
    count = 0
    aux = np.zeros((11,))
    for values in predicts:
        if values >= 0.50:
            aux[count] = 1.
        else:
            aux[count] = 0.
        count += 1
    predict[count_global] = aux
    count_global += 1

In [ ]:
print('Matriz de Confusão:\n', confusion_matrix(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Acurácia:', accuracy_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Precisão', precision_score(y.argmax(axis = 1), predict.argmax(axis = 1)))
print('Sensibilidade:', recall_score(y.argmax(axis = 1), predict.argmax(axis = 1))) 
print('F1_Score:', f1_score(y.argmax(axis = 1), predict.argmax(axis = 1)))

In [ ]:
# visualizando o ganho de acurácia durante o treinamento
plt.figure(figsize = (15,5))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-accuracy')

# visualizando o decaimento da função de custo durante o treinamento 
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('model-loss')

### Visualizando a arquitetura da rede

In [ ]:
# visualizando a arquitetura do modelo
model.summary()